In [ ]:
import numpy as np
from scipy.signal import find_peaks, find_peaks_cwt
import soundfile as sf
from matplotlib import pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (15,4)

In [ ]:
def printShift(shift):
    """
    param shift: should be in second
    """
    shiftInDay = 24*3600*shift
    print("Shift: {} us".format(shift*1000000))
    print("Shift in day: {} s".format(shiftInDay))

In [ ]:
def smartPeakFinder(array, tresholdVal, distance, startOffset=24):
    peaks = []
    lastPeakIndex = None
    for i, d in enumerate(array):
        if i > startOffset:
            if lastPeakIndex == None or (i - lastPeakIndex > distance):
                if d > tresholdVal:
                    lastPeakIndex = i
                    peaks.append(i)
    return peaks

In [ ]:
def calculateAccuracy(filename, plot=True):
    data, samplerate = sf.read(filename)
    if len(np.shape(data)) > 1: # Streo check
        ch1 = data[:,0]
    else:
        ch1 = data
    print("\n\n################")
    print("Filename: {}".format(filename))
    print("Samplerate: {}".format(samplerate))
    print("Min Resolution: {} us".format(1000000*(1.0/samplerate)))
    print("Lenght: {} - {} s".format(len(ch1), float(len(ch1))/samplerate))
    peaks= smartPeakFinder(ch1, 0.005, samplerate-5000)
    peaksTime = [peak*(1.0/samplerate) for peak in peaks]
    
    """
    Method 1
    """
    peaksNo = len(peaks)-1
    shift = ((peaksTime[peaksNo] - peaksTime[0]) - peaksNo) / float(peaksNo)
    printShift(shift)
    """
    Method 2
    """
    try:
        shiftList = []
        sumS = 0
        for i in range(0,len(peaksTime)-1):
            s = peaksTime[i+1] -1 -peaksTime[i] 
            #print(s)
            if np.abs(s) < 0.01:
                sumS += s
                shiftList.append( s )
        shift = sumS / float(len(shiftList))
        #print(shiftList)
        printShift(shift)
    except:
        pass
    if plot==True:
        #fftPlot(ch1,samplerate)
        plt.figure()
        plt.plot(ch1)
        plt.plot(peaks, ch1[peaks], "x")
    return shiftList
    
    

In [ ]:
def fftPlot(ch1, samplerate):
    freq = np.arange(0,samplerate, samplerate/len(ch1))
    plt.figure()
    plt.plot(freq, np.fft.fft(ch1))
    #plt.xlim(0,100)


In [ ]:
calculateAccuracy("test/test5.wav", plot=True)
calculateAccuracy("test/test6.wav", plot=True)
calculateAccuracy("test/test7.wav", plot=True)
calculateAccuracy("test/test8.wav", plot=True)
calculateAccuracy("test/test9.wav", plot=True)
calculateAccuracy("test/test10.wav", plot=True)

In [ ]:
calculateAccuracy("test/test1.wav", plot=True)
calculateAccuracy("test/test2.wav", plot=True)
calculateAccuracy("test/test3.wav", plot=True)
calculateAccuracy("test/test4.wav", plot=True)

In [ ]:
calculateAccuracy("test/test7.wav")

In [ ]:
calculateAccuracy("test/test1-long.wav")